# Question-Answering with BERT



Pre-trained language representations have been shown to improve many downstream NLP tasks such as question answering and natural language inference (NLI). Devlin, Jacob, et al proposed `BERT` [1] (Bidirectional Encoder Representations from Transformers), which fine-tunes deep bi-directional representations on a wide range of tasks with minimal task-specific parameters, and obtained state-of-the-art results.

In this tutorial, you will focus on adapting the `BERT` model for the question answering task on the `SQuAD` dataset. Specifically, you will:

- Understand how to pre-process the `SQuAD` dataset to leverage the learnt representations in `BERT`.
- Adapt the `BERT` model to the question-answering task.
- Load a trained model to perform inference on the `SQuAD` dataset.


## Preparation

Firstly, you need to import the libraries.

In [1]:
import collections, time, logging
import numpy as np
import gluonnlp as nlp
import mxnet as mx
import bert
import qa_utils
from bert.bert_qa_evaluate import PredResult, predict

## Inspecting the `SQuAD` dataset

To understand the task in front of you, you should analyze the Stanford Question Answering Dataset (`SQuAD`). The dataset can be downloaded using the `nlp.data.SQuAD` API. In this tutorial, you will create a small dataset with 3 samples from the `SQuAD` dataset for demonstration purpose.

The question-answering task on the `SQuAD` dataset is setup the following way. For each sample in the dataset, a context is provided. The context is usually a long paragraph which contains lots of information. Then a question is asked based on the context. The goal is to find the text span in the context that answers the question in the sample.

In [2]:
full_data = nlp.data.SQuAD(segment='dev', version='1.1')
# loading a subset of the dev set of SQuAD
num_target_samples = 3
target_samples = [full_data[i] for i in range(num_target_samples)]
dataset = mx.gluon.data.SimpleDataset(target_samples)
print('Number of samples in the created dataset subsampled from SQuAD = %d'%len(dataset))

Number of samples in the created dataset subsampled from SQuAD = 3


Take a look at a sample from the dataset. In this sample, the question is about the location of the game, with a description about the Super Bowl 50 game as the context. Note that three different answer spans are correct for this question, and they start from index 403, 355 and 355 in the context respectively.

In [3]:
sample = dataset[2]

context_idx = 3

print('\nContext:')
print(sample[context_idx])


Context:
Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [4]:
question_idx = 2
answer_idx = 4
answer_pos_idx = 5

print("\nQuestion")
print(sample[question_idx])
print("\nCorrect Answer Spans")
print(sample[answer_idx])
print("\nAnswer Span Start Indices:")
print(sample[answer_pos_idx])


Question
Where did Super Bowl 50 take place?

Correct Answer Spans
['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]

Answer Span Start Indices:
[403, 355, 355]


## Data pre-processing for QA with `BERT`

Recall that during `BERT` pre-training, it takes a sentence pair as the input, separated by the '`SEP`' special token. For `SQuAD`, you can feed the context-question pair as the sentence pair input. To use `BERT` to predict the starting and ending span of the answer, you can add a classification layer for each token in the context texts, to predict if a token is the start or the end of the answer span. 

![qa](qa.png)

In the next few code blocks, you will work on pre-processing the samples in the `SQuAD` dataset in the desired format with these special separators. 


### Tokenizing

First, use the `get_model` API in `GluonNLP` to get the model definition for `BERT`, and the vocabulary used for the `BERT` model. Note that you should discard the pooler and classifier layers used for the next sentence prediction task, as well as the decoder layers for the masked language model task during the `BERT` pre-training phase. These layers are not useful for predicting the starting and ending indices of the answer span.

In [5]:
bert_model, vocab = nlp.model.get_model('bert_12_768_12',
                                        dataset_name='book_corpus_wiki_en_uncased',
                                        use_classifier=False,
                                        use_decoder=False,
                                        use_pooler=False,
                                        pretrained=False)

Note that there are several special tokens in the vocabulary for `BERT`. In particular, the `[SEP]` token is used for separating the sentence pairs, and the `[CLS]` token is added at the beginning of the sentence pairs. They will be used to pre-process the `SQuAD` dataset later.

In [6]:
print(vocab)

Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")


The second step is to process the samples using the same tokenizer used for BERT, which is provided as the `BERTTokenizer` API in `GluonNLP`. Note that instead of word level and character level representation, `BERT` uses subwords to represent a word, separated by `##`s. 

In the following example, the word `suspending` is tokenized as two subwords (`suspend` and `##ing`), and `numerals` is tokenized as three subwords (`nu`, `##meral`, `##s`).

In [7]:
tokenizer = nlp.data.BERTTokenizer(vocab=vocab, lower=True)

tokenizer("as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals")

['as',
 'well',
 'as',
 'temporarily',
 'suspend',
 '##ing',
 'the',
 'tradition',
 'of',
 'naming',
 'each',
 'super',
 'bowl',
 'game',
 'with',
 'roman',
 'nu',
 '##meral',
 '##s']

### Sentence pair composition

With the tokenizer in place, you are ready to process the question-context texts and compose sentence pairs. The functionality is available via the `SQuADTransform` API. 

In [8]:
transform = bert.data.qa.SQuADTransform(tokenizer, is_pad=False, is_training=False, do_lookup=False)
dev_data, _ = bert.data.qa.preprocess_dataset(dataset, transform)
logging.info('The number of examples after preprocessing:{}'.format(len(dev_data)))

Done! Transform dataset costs 0.11 seconds.


Take a look at the sample after the transformation:

In [9]:
sample = dev_data[2]
print('\nsegment type: ' + str(sample[2]))
print('\ntext length: ' + str(sample[3]))


segment type: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

text length: 168


In [10]:
print('\nsentence pair: ' + str(sample[1]))


sentence pair: ['[CLS]', 'where', 'did', 'super', 'bowl', '50', 'take', 'place', '?', '[SEP]', 'super', 'bowl', '50', 'was', 'an', 'american', 'football', 'game', 'to', 'determine', 'the', 'champion', 'of', 'the', 'national', 'football', 'league', '(', 'nfl', ')', 'for', 'the', '2015', 'season', '.', 'the', 'american', 'football', 'conference', '(', 'afc', ')', 'champion', 'denver', 'broncos', 'defeated', 'the', 'national', 'football', 'conference', '(', 'nfc', ')', 'champion', 'carolina', 'panthers', '24', '–', '10', 'to', 'earn', 'their', 'third', 'super', 'bowl', 'title', '.', 'the', 'game', 'was', 'played', 'on', 'february', '7', ',', '2016', ',', 'at', 'levi', "'", 's', 'stadium', 'in', 'the', 'san', 'francisco', 'bay', 'area', 'at', 'santa', 'clara', ',', 'california', '.', 'as', 'this', 'was', 'the', '50th', 'super', 'bowl', ',', 'the', 'league', 'emphasized', 'the', '"', 'golden', 'anniversary', '"', 'with', 'various', 'gold', '-', 'themed', 'initiatives', ',', 'as', 'well', '

### Vocabulary lookup

Finally, convert the transformed texts to subword indices, which are used to contructor NDArrays as the inputs to the model.

In [11]:
def vocab_lookup(example_id, subwords, type_ids, length, start, end):
    indices = vocab[subwords]
    return example_id, indices, type_ids, length, start, end

dev_data = dev_data.transform(vocab_lookup, lazy=False)
print(dev_data[2][1])

[2, 2073, 2106, 3565, 4605, 2753, 2202, 2173, 1029, 3, 3565, 4605, 2753, 2001, 2019, 2137, 2374, 2208, 2000, 5646, 1996, 3410, 1997, 1996, 2120, 2374, 2223, 1006, 5088, 1007, 2005, 1996, 2325, 2161, 1012, 1996, 2137, 2374, 3034, 1006, 10511, 1007, 3410, 7573, 14169, 3249, 1996, 2120, 2374, 3034, 1006, 22309, 1007, 3410, 3792, 12915, 2484, 1516, 2184, 2000, 7796, 2037, 2353, 3565, 4605, 2516, 1012, 1996, 2208, 2001, 2209, 2006, 2337, 1021, 1010, 2355, 1010, 2012, 11902, 1005, 1055, 3346, 1999, 1996, 2624, 3799, 3016, 2181, 2012, 4203, 10254, 1010, 2662, 1012, 2004, 2023, 2001, 1996, 12951, 3565, 4605, 1010, 1996, 2223, 13155, 1996, 1000, 3585, 5315, 1000, 2007, 2536, 2751, 1011, 11773, 11107, 1010, 2004, 2092, 2004, 8184, 28324, 2075, 1996, 4535, 1997, 10324, 2169, 3565, 4605, 2208, 2007, 3142, 16371, 28990, 2015, 1006, 2104, 2029, 1996, 2208, 2052, 2031, 2042, 2124, 2004, 1000, 3565, 4605, 1048, 1000, 1007, 1010, 2061, 2008, 1996, 8154, 2071, 14500, 3444, 1996, 5640, 16371, 28990, 2015

## Defining the model

After the data is processed, you can define the model that uses the representation produced by BERT for predicting the starting and ending positions of the answer span.

In [12]:
class BertForQA(mx.gluon.Block):
    """Model for SQuAD task with BERT.

    Parameters
    ----------
    bert: BERTModel
        Bidirectional encoder with transformer.
    """
    def __init__(self, bert, prefix=None, params=None):
        super(BertForQA, self).__init__(prefix=prefix, params=params)
        self.bert = bert
        with self.name_scope():
            self.span_classifier = mx.gluon.nn.Dense(units=2, flatten=False)

    def forward(self, inputs, token_types, valid_length=None):  # pylint: disable=arguments-differ
        """Generate the unnormalized score for the given the input sequences.

        Parameters
        ----------
        inputs : NDArray, shape (batch_size, seq_length)
            Input words for the sequences.
        token_types : NDArray, shape (batch_size, seq_length)
            Token types for the sequences, used to indicate whether the word belongs to the
            first sentence or the second one.
        valid_length : NDArray or None, shape (batch_size,)
            Valid length of the sequence. This is used to mask the padded tokens.

        Returns
        -------
        outputs : NDArray
            Shape (batch_size, seq_length, 2)
        """
        # Use self.bert to get the representation for each token.
        bert_output = self.bert(inputs, token_types, valid_length)
        
        # Use self.span_classifier to predict the start and end spans
        output = self.span_classifier(bert_output)

        # Use mx.nd.split to split the prediction of start and end spans
        pred_start, pred_end = mx.nd.split(output, axis=2, num_outputs=2)
        return pred_start, pred_end

Now download a BERT model trained on the SQuAD dataset, and prepare the `DataLoader`.

In [13]:
net = BertForQA(bert_model)
ctx = mx.gpu()
ckpt = qa_utils.download_qa_ckpt()
net.load_parameters(ckpt, ctx=ctx)

batch_size = 1
dev_dataloader = mx.gluon.data.DataLoader(
    dev_data, batch_size=batch_size, shuffle=False)

Downloaded checkpoint to ./temp/bert_qa-7eb11865.params


In [14]:
all_results = []

total_num = 0
for data in dev_dataloader:
    example_ids, inputs, token_types, valid_length, _, _ = data
    total_num += len(inputs)
    pred_start, pred_end = net(inputs.astype('float32').as_in_context(ctx),
                               token_types.astype('float32').as_in_context(ctx),
                               valid_length.astype('float32').as_in_context(ctx))

    all_results.append((example_ids, pred_start, pred_end))

And lastly, take a look at the predictions your model can make.

In [15]:
top_results = qa_utils.predict(dataset, all_results, vocab)
first_sample_result = top_results[0]
print('\nContext: %s\n'%(' '.join(first_sample_result['context'])))
print('Question: %s\n'%(' '.join((first_sample_result['question']))))
print('Top prediction: %.2f%% \t %s'%(first_sample_result['prediction'][1] * 100, first_sample_result['prediction'][0]))


Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.

Question: which nfl team represented the afc at super bowl 50 ?

Top prediction: 99.36% 	 Denver Broncos
